In [ ]:
import re 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, WebDriverException
import time
import os

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

search_query = "유니마카롱"
file_name = f"{search_query.replace(' ', '_')}_reviews_google.csv"
# 저장 경로 설정
folder_path = "C:/fintech_service/Fintech_project/google"  # 저장할 폴더 경로를 지정
full_path = f"{folder_path}/{file_name}"

# 디렉토리가 없는 경우 생성
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"디렉토리가 생성되었습니다: {folder_path}")


# 구글 맵 페이지 열기 및 '뱅뱅막국수' 검색
driver.get("https://www.google.com/maps")
try:
    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

In [ ]:
# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()

In [ ]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# 리뷰 섹션의 컨테이너를 찾음
try:
    review_container = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde'))
    )
    
    # 스크롤을 반복적으로 내리기
    max_wait_time = 5  # 최대 대기 시간 시작 (초)
    while True:
        # 현재 로드된 리뷰 수 확인
        reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        previous_count = len(reviews)
        
        # 스크롤을 아래로 내림
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_container)
        
        # 새로운 리뷰가 로드될 때까지 대기
        try:
            WebDriverWait(driver, max_wait_time).until(
                lambda driver: len(driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')) > previous_count
            )
            # 리뷰가 로드되면 대기 시간을 초기화
            max_wait_time = 5
        except TimeoutException:
            # 새로운 리뷰가 로드되지 않으면 더 이상 로드할 리뷰가 없는 것으로 간주하고 종료
            print("더 이상 로드할 리뷰가 없습니다.")
            break

        # 스크롤이 아래로 내려갈수록 로딩 시간이 길어질 수 있으므로, 실패 시 대기 시간 증가
        max_wait_time += 1  # 한 번 실패할 때마다 대기 시간을 1초씩 늘림 (필요 시 조정 가능)

    print("모든 리뷰를 끝까지 로드했습니다.")
except Exception as e:
    print(f"스크롤 중 오류 발생: {e}")


In [ ]:







# 리뷰 데이터 수집
reviews_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
for review in reviews:
    try:
        # '자세히' 버튼 클릭
        try:
            more_button = review.find_element(By.XPATH, './/span[2]/button')
            more_button.click()
            time.sleep(1)  # '자세히' 클릭 후 리뷰 로드 대기
        except NoSuchElementException:
            pass  # '자세히' 버튼이 없으면 무시하고 진행
        
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
        reviews_list.append({
            "Date": date,
            "Review": review_text
        })
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

# 데이터프레임으로 변환 및 출력
reviews_df = pd.DataFrame(reviews_list)


# CSV 파일로 저장
reviews_df.to_csv(full_path, index=False, encoding='utf-8-sig')
print(f"리뷰 데이터가 {full_path}에 저장되었습니다.")

# 드라이버 종료
#driver.quit()